<a href="https://colab.research.google.com/github/vedant-singh-007/DeepFake-Detection-Using-ResNext-50-LSTM/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd
from glob import glob


In [9]:
import os

# Adjust the path based on actual folder location
fake_folder = '/content/drive/MyDrive/fake_vids'
real_folder = '/content/drive/MyDrive/real_vids'

# Check if paths exist
print("Fake folder exists:", os.path.exists(fake_folder))
print("Real folder exists:", os.path.exists(real_folder))


Fake folder exists: True
Real folder exists: True


In [10]:
from glob import glob

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

print("Number of fake videos:", len(fake_videos))
print("Number of real videos:", len(real_videos))


Number of fake videos: 53
Number of real videos: 53


In [11]:
all_videos =  real_videos+fake_videos
labels = [0]*len(real_videos) + [1]*len(fake_videos)  # 1 = Fake, 0 = Real

# Example:
for video, label in zip(all_videos, labels):
    print(f"{video} -> {'Fake' if label == 1 else 'Real'}")



/content/drive/MyDrive/real_vids/Copy of v1.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v10.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v11.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v12.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v13.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v14.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v15.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v16.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v17.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v18.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v19.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v2.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v20.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v21.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v22.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v23.mp4 -> Real
/content/drive/MyDrive/real_vids/Copy of v24.mp4 -> Real
/content/drive/MyDrive/real_vids/

In [12]:
frame_count = []
for video_file in all_videos:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames are  [81, 109, 81, 101, 80, 84, 64, 78, 100, 101, 95, 100, 95, 77, 174, 100, 98, 101, 118, 101, 84, 125, 120, 74, 171, 114, 118, 92, 100, 92, 96, 150, 90, 94, 98, 103, 101, 101, 95, 101, 100, 92, 96, 77, 116, 98, 116, 103, 109, 80, 94, 78, 115, 82, 110, 82, 101, 81, 85, 64, 79, 100, 102, 96, 100, 96, 78, 175, 100, 98, 102, 119, 102, 85, 127, 120, 74, 171, 114, 118, 92, 100, 92, 97, 150, 90, 94, 98, 104, 102, 102, 96, 102, 100, 92, 96, 79, 116, 99, 116, 104, 110, 81, 94, 79, 115]
Total no of video:  106
Average frame per video: 100.86792452830188


In [13]:
output_video_folder = "/content/drive/MyDrive/cropped_face_videos"
os.makedirs(output_video_folder, exist_ok=True)

label_file_path = os.path.join(output_video_folder, "labels.csv")
label_entries = []


In [15]:
import os
import cv2

# Load Haar cascade once globally (adjust the path if needed)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def process_video_to_output(video_path, output_folder, label, max_frames=100, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    video_name = os.path.splitext(os.path.basename(video_path))[0]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) > 0:
            # Pick the largest face
            x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
            face_crop = frame[y:y+h, x:x+w]
            face_crop = cv2.resize(face_crop, size)
            frames.append(face_crop)

    cap.release()

    if len(frames) == 0:
        print(f"Skipping: {video_name} (no face detected)")
        return None

    if len(frames) < max_frames:
        last_frame = frames[-1]
        frames += [last_frame] * (max_frames - len(frames))
    else:
        frames = frames[:max_frames]

    output_path = os.path.join(output_folder, f"{video_name}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 25.0, size)

    for frame in frames:
        out.write(frame)
    out.release()

    return video_name, label


In [16]:
from tqdm import tqdm
import cv2
import os
from glob import glob
from cv2 import CascadeClassifier


fake_folder = '/content/drive/MyDrive/fake_vids'
real_folder = '/content/drive/MyDrive/real_vids'

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

all_video_paths = [(path, 1) for path in fake_videos] + [(path, 0) for path in real_videos]



for path, label in tqdm(all_video_paths):
    result = process_video_to_output(path, output_video_folder, label)
    if result:
        video_name, label = result
        label_entries.append((video_name, label))



100%|██████████| 106/106 [24:19<00:00, 13.77s/it]


In [17]:

df_labels = pd.DataFrame(label_entries, columns=['video_name', 'label'])
df_labels.to_csv(label_file_path, index=False)

In [18]:
df_labels

,video_name,label
0,Copy of vs1,1
1,Copy of vs10,1
2,Copy of vs11,1
3,Copy of vs12,1
4,Copy of vs13,1
...,...,...
101,Copy of v53,0
102,Copy of v6,0
103,Copy of v7,0
104,Copy of v8,0


In [19]:
df_labels = df_labels.drop_duplicates(subset='video_name', keep='first')

# Save to CSV (overwrite if needed)
df_labels.to_csv(label_file_path, index=False)

print(f"Duplicates removed. Final number of videos: {len(df_labels)}")

Duplicates removed. Final number of videos: 106


In [20]:
df_labels

,video_name,label
0,Copy of vs1,1
1,Copy of vs10,1
2,Copy of vs11,1
3,Copy of vs12,1
4,Copy of vs13,1
...,...,...
101,Copy of v53,0
102,Copy of v6,0
103,Copy of v7,0
104,Copy of v8,0


In [21]:
X=df_labels['video_name']
y=df_labels['label']

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [23]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [24]:
type(X_train)

pandas.core.series.Series

In [25]:
X_train

,video_name
0,Copy of v18
1,Copy of vs33
2,Copy of vs3
3,Copy of vs38
4,Copy of vs50
...,...
79,Copy of v26
80,Copy of vs22
81,Copy of v45
82,Copy of vs8


In [26]:
type(X_train)

pandas.core.series.Series

In [27]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [28]:
video_label_dict = dict(zip(df_labels['video_name'], df_labels['label']))


In [29]:
video_label_dict

{'Copy of vs1': 1,
 'Copy of vs10': 1,
 'Copy of vs11': 1,
 'Copy of vs12': 1,
 'Copy of vs13': 1,
 'Copy of vs14': 1,
 'Copy of vs15': 1,
 'Copy of vs16': 1,
 'Copy of vs17': 1,
 'Copy of vs18': 1,
 'Copy of vs19': 1,
 'Copy of vs2': 1,
 'Copy of vs20': 1,
 'Copy of vs21': 1,
 'Copy of vs22': 1,
 'Copy of vs23': 1,
 'Copy of vs24': 1,
 'Copy of vs25': 1,
 'Copy of vs26': 1,
 'Copy of vs27': 1,
 'Copy of vs28': 1,
 'Copy of vs29': 1,
 'Copy of vs3': 1,
 'Copy of vs30': 1,
 'Copy of vs31': 1,
 'Copy of vs32': 1,
 'Copy of vs33': 1,
 'Copy of vs34': 1,
 'Copy of vs35': 1,
 'Copy of vs36': 1,
 'Copy of vs37': 1,
 'Copy of vs38': 1,
 'Copy of vs39': 1,
 'Copy of vs4': 1,
 'Copy of vs40': 1,
 'Copy of vs41': 1,
 'Copy of vs42': 1,
 'Copy of vs43': 1,
 'Copy of vs44': 1,
 'Copy of vs45': 1,
 'Copy of vs46': 1,
 'Copy of vs47': 1,
 'Copy of vs48': 1,
 'Copy of vs49': 1,
 'Copy of vs5': 1,
 'Copy of vs50': 1,
 'Copy of vs51': 1,
 'Copy of vs52': 1,
 'Copy of vs53': 1,
 'Copy of vs6': 1,
 'Copy

In [30]:
import torchvision.transforms as T
from torch.utils.data import Dataset
import torch
import numpy as np
import cv2
import os

class VideoDataSet(Dataset):
    def __init__(self, video_paths, label_dict, sequence_length=100):
        self.video_paths = video_paths
        self.label_dict = label_dict
        self.seq_length = sequence_length
        self.transform = T.Compose([
            T.ToPILImage(),                # Convert np.array to PIL
            T.Resize((224, 224)),          # Resize frame
            T.ToTensor(),                  # Convert to tensor [C, H, W], scaled to [0,1]
        ])

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, index):
        path = self.video_paths[index]
        video_name = os.path.splitext(os.path.basename(path))[0]
        label = self.label_dict.get(video_name, -1)
        frames = self.extract_frames(path)

        if len(frames) == 0:
            # fallback to dummy black frames
            dummy = torch.zeros((3, 224, 224))
            frames = [dummy] * self.seq_length

        video_tensor = torch.stack(frames)  # [seq_len, 3, 224, 224]
        return video_tensor, label

    def extract_frames(self, path):
        cap = cv2.VideoCapture(path)
        frames = []
        while len(frames) < self.seq_length:
            success, frame = cap.read()
            if not success:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            frame_tensor = self.transform(frame)  # [3, 224, 224]
            frames.append(frame_tensor)

        # Pad with black frames if not enough
        while len(frames) < self.seq_length:
            frames.append(torch.zeros((3, 224, 224)))

        cap.release()
        return frames





In [31]:
type(X_train)

pandas.core.series.Series

In [32]:
X_train_dataset=VideoDataSet(X_train,video_label_dict,sequence_length=100)
X_test_dataset=VideoDataSet(X_test,video_label_dict,sequence_length=100)

In [33]:
X_train_dataset

In [34]:
type(X_train_dataset)

__main__.VideoDataSet

In [35]:
y_train_dataset=DataLoader(X_train_dataset,batch_size=12,shuffle=True)
y_test_dataset=DataLoader(X_test_dataset,batch_size=2,shuffle=True)

In [36]:
type(y_train_dataset)

torch.utils.data.dataloader.DataLoader

In [37]:
from torch.utils.data import DataLoader

train_loader = DataLoader(X_train_dataset, batch_size=2, shuffle=True)

for videos, labels in train_loader:
    print("Batch video tensor shape:", videos.shape)
    print("Batch labels:", labels)





Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor sh

In [38]:
test_loader=DataLoader(X_test_dataset,batch_size=2,shuffle=True)
for videos, labels in test_loader:
    print("Batch video tensor shape:", videos.shape)
    print("Batch labels:", labels)


Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([1, 0])
Batch video tensor shape: torch.Size([2, 100, 3, 224, 224])
Batch labels: tensor([0, 1])


In [39]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNeXtLSTMModel(nn.Module):
    def __init__(self, hidden_size=512, num_layers=1, dropout=0.0):
        super(ResNeXtLSTMModel, self).__init__()

        # Load pretrained ResNeXt50 and remove final FC layer
        resnext = models.resnext50_32x4d(pretrained=True)
        self.resnext_backbone = nn.Sequential(*list(resnext.children())[:-1])  # outputs [B*T, 2048, 1, 1]

        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=2048,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: [B, T, 3, 224, 224]
        B, T, C, H, W = x.shape

        # Flatten batch and time dims to feed frames into resnext
        x = x.view(B * T, C, H, W)  # [B*T, 3, 224, 224]

        features = self.resnext_backbone(x)  # [B*T, 2048, 1, 1]
        features = features.view(B, T, 2048)  # [B, T, 2048]

        lstm_out, (hn, cn) = self.lstm(features)  # lstm_out: [B, T, hidden_size]

        # Take the output of the last timestep
        last_out = lstm_out[:, -1, :]  # [B, hidden_size]

        out = self.classifier(last_out)  # [B, 1], sigmoid output

        return out


In [40]:
model = ResNeXtLSTMModel(hidden_size=512, num_layers=1)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [41]:
import torch
import torch.nn.functional as F

def train_model(model, train_loader, num_epochs, optimizer, device):
    model.to(device)
    model.train()

    criterion = torch.nn.BCELoss()  # since output is sigmoid prob

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0

        print(f"Epoch {epoch + 1}/{num_epochs}")

        for i, (videos, labels) in enumerate(train_loader):
            videos = videos.to(device)              # [B, T, 3, 224, 224]
            labels = labels.float().to(device)      # labels: [B], binary 0 or 1

            optimizer.zero_grad()

            outputs = model(videos)                  # [B, 1]
            outputs = outputs.squeeze(1)             # [B]

            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * videos.size(0)

            preds = (outputs >= 0.5).float()        # Thresholding at 0.5
            running_corrects += torch.sum(preds == labels).item()
            total_samples += videos.size(0)

            if (i + 1) % 10 == 0 or (i + 1) == len(train_loader):
                print(f"Batch {i + 1}/{len(train_loader)} - "
                      f"Loss: {loss.item():.4f} - "
                      f"Acc: {running_corrects / total_samples:.4f}")

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        print(f"Epoch {epoch + 1} complete. Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}\n")


In [1]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ResNeXtLSTMModel(hidden_size=512, num_layers=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# assuming train_loader is ready with (videos, labels)

train_model(model, train_loader, num_epochs=10, optimizer=optimizer, device=device)


NameError: name 'torch' is not defined